In [35]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils import GC
import time

In [36]:
""" Define functions in this block """
#This function is a sliding window counting function for any nucleotide substring.
#Inputs: nucleotide query (str), nucleotide reference (str)
#Outputs: number of occurrences of query in reference in seq (int)
def nuc_count(nuc, seq):
    """ Initialize variables """
    nuc_locs = []
    my_bool = True
    start = 0
    stop = len(seq)

    """ Perform count """
    while my_bool:
        a = seq.find(nuc, start, stop)
        if a != -1: ###If search finds a value
            nuc_locs.append(a) ###Append location of match to list
            start = a+1 ###Iterate starting location for search
        else:
            my_bool = False ###Break from the loop
    return len(nuc_locs)

#This function counts the frequencies of all 4 nucleotides in a sequence.
#Inputs: nucleotide reference (str)
#Outputs: frequency of each nucleotide (dict)
def nuc_freq(seq):
    """ Initialize variables """
    l = len(seq) ###Get total number of nt
    nuc_f_dict = {} ###Initialize dict

    """ Perform count """
    for char in "ATCG":
        nuc_f_dict[char] = seq.count(char)/l
    return nuc_f_dict

#This function counts the frequencies of all 16 dinucleotides in a sequence.
#Inputs: nucleotide reference (str)
#Outputs: frequency of each dinucleotide (dict)
def dinuc_freq(seq):
    """ Initialize variables """
    l = len(seq) ###Get total number of nt
    dinuc_f_dict = {} ###Initialize dict
    """ Perform count """
    for char1 in "ATCG":
        for char2 in "ATCG":
            dinuc = char1 + char2
            dinuc_f_dict[dinuc] = nuc_count(dinuc,seq)/(l-1)
    return dinuc_f_dict

In [37]:
""" Define inputs in this block """

# file = "T-thermophilusHB8chromosome1.fasta"
file = "baccoa.fna"

In [38]:
""" Gather nuc and dinuc frequencies """

""" Intialize variables """
tic = time.perf_counter() ###Start timer!
ref_seq = "" ###This is in case there are multiple chromosomes/scaffolds/contigs in the FASTA file

records = SeqIO.parse(file, "fasta") ###Create iterator for FASTA file
for record in records:
    # seq = record.seq
    seq = record.seq.__str__().replace("N","") ###Removes any N's from genome
    seq = seq.upper() ###Makes sure sequence is uppercase so the counts aren't messed up
    ref_seq += seq

gc_content = (nuc_count("G",ref_seq) + nuc_count("C",ref_seq))/len(ref_seq) ###Get GC content
nuc_dict = nuc_freq(ref_seq) ###Get nucleotide freqs
dinuc_dict = dinuc_freq(ref_seq) ###Get dinucleotide freqs
toc = time.perf_counter() ###Stop timer!

In [39]:
""" Output results to a TXT file """

"""Initialize variables """
output_file = "test.txt"

""" Write to output TXT file """
with open(output_file,'w+') as f:
    f.write("GC CONTENT:\n" + str(gc_content) + "\n")
    f.write("\nNUCLEOTIDE FREQUENCIES:\n")
    for nuc in nuc_dict: ###Print nucleotide freqs
        f.write(nuc + ": " + str(nuc_dict[nuc]) + "\n")
    f.write("\nDINUCLEOTIDE FREQUENCIES:\n")
    for dinuc in dinuc_dict: ###Print dinucleotide frequencies
        f.write(dinuc + ": " + str(dinuc_dict[dinuc]) + "\n")
    f.write("\nCode executed in %0.4f seconds" % (toc-tic))